In [1]:
import pandas as pd  
import numpy as np
from pandas_datareader import data, wb
import datetime
from dateutil.relativedelta import relativedelta
import scipy.optimize as sco
from scipy.optimize import NonlinearConstraint
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

c:\Users\e0023491\AppData\Local\mambaforge\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\e0023491\AppData\Local\mambaforge\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\e0023491\AppData\Local\mambaforge\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
st_df = pd.read_csv('10stocks_1year.csv')
tickers = st_df.iloc[:10,0].to_list()
start = datetime.datetime(2021, 10, 1)
end = datetime.datetime(2022, 10, 31)
df = pd.DataFrame([data.DataReader(ticker, 'yahoo', start, end)['Adj Close'] for ticker in tickers]).T
df.columns = tickers

In [4]:
df

,MKTX,AVGO,MTCH,CMCSA,AMD,TROW,ADBE,BIIB,ABMD,POOL
Date,,,,,,,,,,
2021-09-30,415.811737,470.728760,156.990005,54.246754,102.900002,189.724411,575.719971,282.989990,325.519989,430.708527
2021-10-01,412.975037,473.077911,158.630005,55.488224,102.449997,191.499146,577.469971,283.940002,323.329987,433.673035
2021-10-04,412.975037,462.011780,152.649994,55.071167,100.339996,186.724716,558.489990,278.320007,313.869995,432.205658
2021-10-05,426.526001,471.204468,154.500000,55.324463,101.809998,188.152222,566.700012,280.440002,332.470001,436.657379
2021-10-06,407.647522,474.466034,157.309998,55.431618,103.639999,186.907959,570.309998,283.549988,337.299988,437.192810
...,...,...,...,...,...,...,...,...,...,...
2022-10-25,246.062180,460.070007,45.430000,31.559999,61.470001,108.690002,323.790009,273.559998,264.290009,298.540009
2022-10-26,237.616470,461.179993,44.230000,31.500000,59.730000,109.070000,320.480011,280.429993,268.250000,305.880005
2022-10-27,237.117905,455.399994,43.560001,31.969999,58.599998,106.639999,318.649994,277.869995,267.579987,304.230011


In [6]:
df.to_csv('daily_price.csv')

## write file for matlab to calculate robust estimate of covariance

In [7]:
dailyRet = df.pct_change()
dailyRet.to_csv('daily_return.csv')
dailyRet

,MKTX,AVGO,MTCH,CMCSA,AMD,TROW,ADBE,BIIB,ABMD,POOL
Date,,,,,,,,,,
2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-01,-0.006822,0.004990,0.010447,0.022886,-0.004373,0.009354,0.003040,0.003357,-0.006728,0.006883
2021-10-04,0.000000,-0.023392,-0.037698,-0.007516,-0.020595,-0.024932,-0.032867,-0.019793,-0.029258,-0.003384
2021-10-05,0.032813,0.019897,0.012119,0.004599,0.014650,0.007645,0.014700,0.007617,0.059260,0.010300
2021-10-06,-0.044261,0.006922,0.018188,0.001937,0.017975,-0.006613,0.006370,0.011090,0.014528,0.001226
...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.020259,0.007776,0.067936,0.015444,0.047189,0.041092,0.023939,-0.003860,0.015407,0.019499
2022-10-26,-0.034323,0.002413,-0.026414,-0.001901,-0.028307,0.003496,-0.010223,0.025113,0.014984,0.024586
2022-10-27,-0.002098,-0.012533,-0.015148,0.014921,-0.018918,-0.022279,-0.005710,-0.009129,-0.002498,-0.005394


In [5]:
start = datetime.datetime(2022, 9, 30)
end = datetime.datetime(2022, 10, 31)
df2 = pd.DataFrame([data.DataReader(ticker, 'yahoo', start, end)['Adj Close'] for ticker in tickers]).T
df2.columns = tickers
octRet = (df2.iloc[-1,:] - df2.iloc[0,:]) / df2.iloc[0,:]
np.array(octRet)

array([ 0.08655377,  0.04440937, -0.1046632 ,  0.05232571, -0.06361082,
       -0.01145354,  0.14465409,  0.07249888,  0.01253211, -0.03657606])

## October optimization

In [64]:
test_date = datetime.date(2022, 10, 31) # datetime.date(2022, 8, 31) datetime.date(2022, 9, 29) datetime.date(2022, 10, 31)

file = open('expected_returns.pkl', 'rb')
allReturns = pickle.load(file)
file.close()

actRet = []
predRet = []
for itic in tickers:
    actRet.append(allReturns[test_date][itic]['actual_rtn'])
    predRet.append(allReturns[test_date][itic]['predicted_rtn'])
    
actRet = np.array(octRet)

In [65]:
actRet,predRet

(array([ 0.08655377,  0.04440937, -0.1046632 ,  0.05232571, -0.06361082,
        -0.01145354,  0.14465409,  0.07249888,  0.01253211, -0.03657606]),
 [0.00607628770615265,
  0.00976028221583336,
  -0.09689316532139569,
  -0.07944423390554123,
  -0.06427846164221218,
  -0.03388142736299904,
  -0.022475840527921842,
  -0.023583450718641985,
  -0.0015060728921572064,
  -0.012965447519838302])

In [12]:
def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 12
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

In [66]:
mean_returns = predRet#df.pct_change().mean()
cov = pd.read_csv('robust_cov_upto10.csv', header=None)#df.pct_change().cov()
rf_df = pd.read_csv("US_treasury_1year.csv", index_col = 0)
rf = float(rf_df.loc['9/29/2022'].DGS1)/100

In [67]:
mean_returns, rf

([0.00607628770615265,
  0.00976028221583336,
  -0.09689316532139569,
  -0.07944423390554123,
  -0.06427846164221218,
  -0.03388142736299904,
  -0.022475840527921842,
  -0.023583450718641985,
  -0.0015060728921572064,
  -0.012965447519838302],
 0.0398)

## maximize Sharpe

In [10]:
def calc_neg_sharpe(weights, mean_returns, cov, rf):
    weights = np.append(weights, 1 - sum(weights))
    portfolio_return = np.sum(mean_returns * weights) * 12
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return -sharpe_ratio

def max_sharpe_ratio(mean_returns, cov, rf, flagAllowShort):
    num_assets = len(mean_returns)
    args = (mean_returns, cov, rf)
    if not flagAllowShort:
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bound = (0,1)#(-float('inf'),float('inf'))
        bounds = tuple(bound for asset in range(num_assets-1))
        con = lambda x: np.sum(x)
        nlc = NonlinearConstraint(con, 0, 1)
        result = sco.minimize(calc_neg_sharpe, (num_assets-1)*[1./num_assets,], args=args, bounds=bounds,
                            method='SLSQP', constraints=nlc)
    else:
        bound = (-1,1)#(-float('inf'),float('inf'))
        bounds = tuple(bound for asset in range(num_assets-1))
        con = lambda x: np.sum(x)
        nlc = NonlinearConstraint(con, -1, 1)
        result = sco.minimize(calc_neg_sharpe, (num_assets-1)*[1./num_assets,], args=args, bounds=bounds,
                             method='SLSQP', constraints=nlc)
        
    return result

In [44]:
optimal_port_sharpe = max_sharpe_ratio(np.array(mean_returns), cov, rf, True)
optimal_port_sharpe

     fun: -3.7566431908462827
     jac: array([-3.03141028e-01, -7.28999138e-01,  2.35140324e-05, -1.06450915e-03,
       -1.25646591e-04,  4.09185886e-05, -6.86645508e-05, -1.07198954e-04,
        2.36928463e-05])
 message: 'Optimization terminated successfully'
    nfev: 121
     nit: 12
    njev: 12
  status: 0
 success: True
       x: array([ 1.        ,  1.        , -0.63422649, -0.9967846 , -0.9046994 ,
       -0.49120152,  0.10234179,  0.2405285 ,  0.70790625])

In [45]:
w_opt = np.append(optimal_port_sharpe['x'], 1-sum(optimal_port_sharpe['x']))
print(w_opt)
print(sum(w_opt))

[ 1.          1.         -0.63422649 -0.9967846  -0.9046994  -0.49120152
  0.10234179  0.2405285   0.70790625  0.97613547]
1.0


In [46]:
tmp = pd.DataFrame([round(x,2) for x in w_opt],index=tickers).T
tmp

,MKTX,AVGO,MTCH,CMCSA,AMD,TROW,ADBE,BIIB,ABMD,POOL
0,1.0,1.0,-0.63,-1.0,-0.9,-0.49,0.1,0.24,0.71,0.98


In [27]:
#tmp.to_csv('oct2022.csv')

In [47]:
portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(w_opt, mean_returns, cov, rf)
print(portfolio_return, portfolio_std, sharpe_ratio)
portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(w_opt, actRet, cov, rf)
print(portfolio_return, portfolio_std, sharpe_ratio)

2.5149355638466293 0.6662957956576001 3.7566431908462827
2.565253158250786 0.6662957956576001 3.8321615938319953


In [151]:
def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))*2 - 1
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])
        
    return results_df
    
num_portfolios = 500000
results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)
#locate position of portfolio with highest Sharpe Ratio
max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
#locate positon of portfolio with minimum standard deviation
min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]
#create scatter plot coloured by Sharpe Ratio
plt.subplots(figsize=(15,10))
plt.scatter(results_frame.stdev,results_frame.ret,c=results_frame.sharpe,cmap='RdYlBu')
plt.xlabel('Standard Deviation')
plt.ylabel('Returns')
plt.colorbar()
#plot red star to highlight position of portfolio with highest Sharpe Ratio
plt.scatter(max_sharpe_port[1],max_sharpe_port[0],marker=(5,1,0),color='r',s=500)
#plot green star to highlight position of minimum variance portfolio
#plt.scatter(min_vol_port[1],min_vol_port[0],marker=(5,1,0),color='g',s=500)
plt.show()

"\nnum_portfolios = 500000\nresults_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)\n#locate position of portfolio with highest Sharpe Ratio\nmax_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]\n#locate positon of portfolio with minimum standard deviation\nmin_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]\n#create scatter plot coloured by Sharpe Ratio\nplt.subplots(figsize=(15,10))\nplt.scatter(results_frame.stdev,results_frame.ret,c=results_frame.sharpe,cmap='RdYlBu')\nplt.xlabel('Standard Deviation')\nplt.ylabel('Returns')\nplt.colorbar()\n#plot red star to highlight position of portfolio with highest Sharpe Ratio\nplt.scatter(max_sharpe_port[1],max_sharpe_port[0],marker=(5,1,0),color='r',s=500)\n#plot green star to highlight position of minimum variance portfolio\n#plt.scatter(min_vol_port[1],min_vol_port[0],marker=(5,1,0),color='g',s=500)\nplt.show()\n"

## on training Period

In [13]:
rf_df = pd.read_csv("US_treasury_1year.csv")
allReturns = pd.read_csv('ret_signal_all.csv')
weights_hist = []
perf_hist = []
shortFalg = True
for i in range(allReturns.shape[0]-1):
    
    actRet = []
    predRet = []
    for itic in tickers:
        actRet.append(allReturns[itic+'_actualret'][i])
        predRet.append(allReturns[itic+'_predictret'][i])
        
    if i == allReturns.shape[0]-1:
        actRet = octRet
        
    #if i == 0:
    #    actRet_prev = actRet.copy()
        
    print(allReturns['Date'][i])
    enddate = allReturns['Date'][i]
    enddate = enddate.split('/')
    end = datetime.datetime(int(enddate[2]), int(enddate[0]), int(enddate[1]))
    start = end - relativedelta(months=3)

    df = pd.DataFrame([data.DataReader(ticker, 'yahoo', start, end)['Adj Close'] for ticker in tickers]).T
    df.columns = tickers
    dailyRet = df.pct_change()
    cov = dailyRet.cov()
    
    enddate2 = allReturns['Date'][i+1]
    enddate2 = enddate2.split('/')
    end2 = datetime.datetime(int(enddate2[2]), int(enddate2[0]), int(enddate2[1]))
    df2 = pd.DataFrame([data.DataReader(ticker, 'yahoo', end, end2)['Adj Close'] for ticker in tickers]).T
    df2.columns = tickers
    dailyRet2 = df2.pct_change()
    cov2 = dailyRet2.cov()
    
    print(end,end2)
    
    startDate = int(enddate[1])
    month = int(enddate[0])
    year = int(enddate[2])
    indx = []
    while len(indx) == 0:
        indx = rf_df.index[rf_df['DATE'] == str(month) +'/'+str(startDate) + '/' + str(year)].to_list()
        try:
            float(rf_df.iloc[indx,1])/100
        except:
            indx = []
            
        if month in [1,3,5,7,8,10,12]:
            if startDate == 31:
               startDate = 1
               month += 1
            else:
               startDate += 1
        else:
            if startDate == 30:
               startDate = 1
               month += 1
            else:
               startDate += 1
        
           
    print(rf_df.iloc[indx,:])
    
    rf = float(rf_df.iloc[indx,1])/100
    
    optimal_port_sharpe = max_sharpe_ratio(predRet, cov, rf, shortFalg)
    w_opt = np.append(optimal_port_sharpe['x'], 1-sum(optimal_port_sharpe['x']))
    
    portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(w_opt, actRet, cov2, rf)
    arr = np.append([portfolio_return, portfolio_std, sharpe_ratio, rf], w_opt)
    
    
    print([portfolio_return, portfolio_std, sharpe_ratio])
    
    weights_hist.append(arr)
        
    #actRet_prev = actRet.copy()
            

12/30/2019
2019-12-30 00:00:00 2020-01-30 00:00:00
             DATE  DGS1
15129  12/30/2019  1.57
[-0.23952437622433398, 0.373992133140297, -0.6824324727937279]
1/30/2020
2020-01-30 00:00:00 2020-03-02 00:00:00
            DATE  DGS1
15152  1/30/2020  1.48
[-0.9070909802741693, 0.5346367936109576, -1.7243313428686455]
3/2/2020
2020-03-02 00:00:00 2020-03-31 00:00:00
           DATE  DGS1
15174  3/2/2020  0.89
[-0.8374764029351556, 1.7344733407450235, -0.4879731403491069]
3/31/2020
2020-03-31 00:00:00 2020-04-30 00:00:00
            DATE  DGS1
15195  3/31/2020  0.17
[4.668948064640541, 0.7950925431213807, 5.870068968723844]
4/30/2020
2020-04-30 00:00:00 2020-06-01 00:00:00
            DATE  DGS1
15217  4/30/2020  0.16
[1.652865684918409, 0.9181855774903681, 1.7984008085072878]
6/1/2020
2020-06-01 00:00:00 2020-06-30 00:00:00
           DATE  DGS1
15239  6/1/2020  0.17
[-1.5015845529921397, 0.6310241187933414, -2.3822933358977694]
6/30/2020
2020-06-30 00:00:00 2020-07-30 00:00:00
      

In [ ]:
actRet, predRet, rf

In [15]:
out_df = pd.DataFrame(np.row_stack(weights_hist),columns=['ret','stdev','sharpe','rf'] + tickers)
out_df

,ret,stdev,sharpe,rf,MKTX,AVGO,MTCH,CMCSA,AMD,TROW,ADBE,BIIB,ABMD,POOL
0,-0.239524,0.373992,-0.682432,0.0157,0.241975,1.000000,0.374633,-0.308260,-1.000000,0.361427,0.008515,0.040128,0.078053,2.035298e-01
1,-0.907091,0.534637,-1.724331,0.0148,0.840572,1.000000,0.313542,-0.057344,-1.000000,-0.520361,0.060938,0.169874,0.003667,1.891127e-01
2,-0.837476,1.734473,-0.487973,0.0089,-0.160506,1.000000,0.022629,-0.292718,-0.723193,-0.767759,0.833545,-0.133879,-0.048691,1.270573e+00
3,4.668948,0.795093,5.870069,0.0017,0.805226,-0.170233,-0.135822,1.000000,-1.000000,-0.595577,1.000000,-0.435233,0.531640,8.881784e-16
4,1.652866,0.918186,1.798401,0.0016,-0.291282,0.256663,1.000000,1.000000,0.054143,-1.000000,-1.000000,1.000000,-0.544415,5.248902e-01
5,-1.501585,0.631024,-2.382293,0.0017,1.000000,1.000000,-1.000000,-0.511897,-1.000000,0.511897,1.000000,1.000000,-1.000000,-4.440892e-16
6,-2.786122,1.077404,-2.587443,0.0016,1.000000,1.000000,-1.000000,1.000000,-0.980350,-1.000000,0.494800,-0.269919,0.415610,3.398593e-01
7,0.898909,0.766351,1.171537,0.0011,1.000000,0.792118,-0.390823,1.000000,-0.709053,-0.972212,1.000000,-0.199981,-0.599366,7.931760e-02
8,-0.169830,0.862100,-0.198387,0.0012,0.686228,0.226712,-0.014496,-0.886506,-0.328354,-1.000000,-0.495956,-0.187629,1.000000,2.000000e+00
9,-2.569901,0.445499,-5.771285,0.0012,-1.000000,1.000000,-0.374435,1.000000,-0.250480,-0.512122,1.000000,-0.771460,0.172853,7.356431e-01


In [16]:
if shortFalg:
    out_df.to_csv('out_sample_performance_shortAllow_full.csv')
else:       
    out_df.to_csv('out_sample_performance_shortNotAllow.csv')

## min variance

In [200]:
def calc_portfolio_std(weights, mean_returns, cov):
    weights = np.append(weights, 1 - sum(weights))
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    return portfolio_std

def min_variance(mean_returns, cov, flagAllowShort):
    num_assets = len(mean_returns)
    args = (mean_returns, cov)
    if not flagAllowShort:
        bound = (0,1)
        bounds = tuple(bound for asset in range(num_assets-1))
        con = lambda x: np.sum(x)
        nlc = NonlinearConstraint(con, 0, 1)
        result = sco.minimize(calc_portfolio_std, (num_assets-1)*[1./num_assets,], args=args, bounds=bounds,
                                method='SLSQP', constraints=nlc)
    else:
        bound = (-float('inf'),float('inf'))
        bounds = tuple(bound for asset in range(num_assets-1))
        result = sco.minimize(calc_portfolio_std, (num_assets-1)*[1./num_assets,], args=args, bounds=bounds)
        
    return result
min_port_variance = min_variance(mean_returns, cov, True)
min_port_variance

      fun: 0.17419777142273796
 hess_inv: <9x9 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 6.79734047e-06, -2.03975725e-05,  3.08086889e-07, -1.70391479e-05,
        8.12128142e-06, -1.02667874e-05,  1.25371934e-05,  8.74300632e-07,
        2.05668815e-06])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 180
      nit: 15
     njev: 18
   status: 0
  success: True
        x: array([ 0.02843546,  0.09272579, -0.06160482,  0.11848163, -0.05870634,
        0.35147167,  0.52053487, -0.00405531, -0.05575689])

In [201]:
w_opt = np.append(min_port_variance['x'], 1-sum(min_port_variance['x']))
print(w_opt)
print(sum(w_opt))
pd.DataFrame(w_opt,index=tickers).T

[ 0.02843546  0.09272579 -0.06160482  0.11848163 -0.05870634  0.35147167
  0.52053487 -0.00405531 -0.05575689  0.06847394]
1.0


,BKNG,AVGO,KLAC,MKTX,TECH,WTW,AMGN,NFLX,TER,REGN
0,0.028435,0.092726,-0.061605,0.118482,-0.058706,0.351472,0.520535,-0.004055,-0.055757,0.068474


In [202]:
portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(np.array(w_opt), mean_returns, cov, rf)
portfolio_return, portfolio_std, sharpe_ratio

(0.12339478037757326, 0.17419777142273796, 0.6400471112055794)

In [203]:
portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(np.array(w_opt), actRet, cov, rf)
print(portfolio_return, portfolio_std, sharpe_ratio)

-0.12488239558320682 0.17419777142273796 -0.7852132347391941
